<a href="https://colab.research.google.com/github/Nobobi-Hasan/FND-Llama/blob/main/FND_Llama_01_Fine_Tuning_1_00_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# --- 1: Mount Google Drive ---

In [1]:
# to save the trained model permanently
from google.colab import drive
drive.mount('/content/drive')

# Create the project directory if it doesn't exist
!mkdir -p /content/drive/MyDrive/ML-Models/FND_Llama_01_Fine_Tuning_1.00

Mounted at /content/drive


# --- 2: Install Dependencies ---

In [2]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets

In [3]:
!pip install -q accelerate peft bitsandbytes transformers trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 43.4 MB/s eta 0:00:00


# --- 3: Hugging Face Login ---

In [4]:
from huggingface_hub import login
login()

# --- 4: Imports ---

In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# --- 5: Model & Dataset Parameters ---

In [6]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

dataset_name = "liar"

# Fine-tuned model name (for temporary adapter storage)
new_model_adapter = "Llama-3.2-3B-fake-news-adapter"

# Permanent model path in Google Drive
output_model_path = "/content/drive/MyDrive/my_research_project/Llama-3.2-3B-fake-news-classifier-merged"

In [7]:
# QLoRA parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

In [8]:
# bitsandbytes parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

In [9]:
# TrainingArguments parameters
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 2 # Reduced for T4 stability
per_device_eval_batch_size = 2
gradient_accumulation_steps = 2 # Increased to compensate for batch size
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25


In [10]:
# SFT parameters
max_seq_length = None
packing = False
device_map = {"": 0}

# --- 6: Data Preprocessing ---

In [12]:

# Format the LIAR dataset into an instruction format for SFT
def format_instruction(sample):
    # Simplify to Real (True, Mostly-true) vs. Fake (all others)
    label = "Real" if sample['label'] in [2, 3] else "Fake"

    # Use the Llama 3.2 Instruct chat template
    prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nClassify the truthfulness of the following statement: \"{sample['statement']}\"\n\nRespond with only one word: Real or Fake.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{label}<|eot_id|>"
    return {"text": prompt}

# Load dataset
dataset = load_dataset(dataset_name, split="train", trust_remote_code=True)

# Apply formatting
dataset = dataset.map(format_instruction, remove_columns=list(dataset.features))

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'liar' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'liar' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


RuntimeError: Dataset scripts are no longer supported, but found liar.py